In [1]:
# -*- coding: utf-8
# Reinaldo Chaves (@paidatocandeira)
# Cria dataframes únicos para documentos de empenho e pagamento
# Faz cruzamentos para encontrar informaçõs dos fornecedores
# Filtra os gastos de combate da covid-19, que possuem 
# ['Código Ação'] == '21C0'

In [2]:
import pandas as pd
from pathlib import Path
import os

# Documentos de empenho, liquidação e pagamento

In [3]:
# Os zips de cada dia podem ser encontrados aqui: http://transparencia.gov.br/download-de-dados/despesas

In [4]:
# Em cada zip os arquivos que vamos usar são os que terminam por:

# Despesas_Empenho

# Despesas_Pagamento

# Despesas_Pagamento_EmpenhosImpactados

### Agrupa todos os arquivos de Despesas_Empenho em único dataframe

In [5]:
# Carrega as despesas empenhadas

In [6]:
print(os.getcwd())

/home/abraji/Documentos/Code/covid_19/repo/portal_transparencia


In [7]:
def _reader_pasta(fname):
        return pd.read_csv(fname, sep=';', encoding='latin_1', dtype = 'str')

In [8]:
folder = Path("zips_empenho_liquidacao_e_pagamento/Despesas_Empenho")
despesas_empenho = pd.concat([
    _reader_pasta(csv)
    for csv in folder.glob("*.csv")
])

In [9]:
despesas_empenho.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1117690 entries, 0 to 8143
Data columns (total 63 columns):
 #   Column                                            Non-Null Count    Dtype 
---  ------                                            --------------    ----- 
 0   Id Empenho                                        1117690 non-null  object
 1   Código Empenho                                    1117690 non-null  object
 2   Código Empenho Resumido                           1117690 non-null  object
 3   Data Emissão                                      1117690 non-null  object
 4   Código Tipo Documento                             1117690 non-null  object
 5   Tipo Documento                                    1117690 non-null  object
 6   Tipo Empenho                                      1117690 non-null  object
 7   Espécie Empenho                                   1117690 non-null  object
 8   Código Órgão Superior                             1117690 non-null  object
 9   Órgão

### Separa pelo código do coronavírus

In [10]:
despesas_empenho_covid19 = despesas_empenho[(despesas_empenho['Código Ação'] == '21C0')]
despesas_empenho_covid19.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22484 entries, 1 to 7493
Data columns (total 63 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Id Empenho                                        22484 non-null  object
 1   Código Empenho                                    22484 non-null  object
 2   Código Empenho Resumido                           22484 non-null  object
 3   Data Emissão                                      22484 non-null  object
 4   Código Tipo Documento                             22484 non-null  object
 5   Tipo Documento                                    22484 non-null  object
 6   Tipo Empenho                                      22484 non-null  object
 7   Espécie Empenho                                   22484 non-null  object
 8   Código Órgão Superior                             22484 non-null  object
 9   Órgão Superior               

### Agrupa todos os arquivos de Despesas_Pagamento_EmpenhosImpactados em único dataframe
### Carrega todos pagamentos de empenhos impactados 

In [11]:
# O problema é que o pagamento de um empenho pode ser numa data diferente
# Por isso é preciso baixar um grande intervalo de arquivos

In [12]:
folder = Path("zips_empenho_liquidacao_e_pagamento/Despesas_Pagamento_EmpenhosImpactados")
despesas_pagamento_empenhosimpactados = pd.concat([
    _reader_pasta(csv)
    for csv in folder.glob("*.csv")
])

In [13]:
despesas_pagamento_empenhosimpactados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3281736 entries, 0 to 30150
Data columns (total 8 columns):
 #   Column                               Dtype 
---  ------                               ----- 
 0   Código Pagamento                     object
 1   Código Empenho                       object
 2   Código Natureza Despesa Completa     object
 3   Subitem                              object
 4   Valor Pago (R$)                      object
 5   Valor Restos a Pagar Inscritos (R$)  object
 6   Valor Restos a Pagar Cancelado (R$)  object
 7   Valor Restos a Pagar Pagos (R$)      object
dtypes: object(8)
memory usage: 225.3+ MB


### Cruza os dois dataframes pelo "Código Empenho"

In [14]:
despesas_pagamentos_empenho_covid19 = pd.merge(despesas_empenho_covid19, despesas_pagamento_empenhosimpactados, left_on='Código Empenho', right_on='Código Empenho')
despesas_pagamentos_empenho_covid19.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18455 entries, 0 to 18454
Data columns (total 70 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Id Empenho                                        18455 non-null  object
 1   Código Empenho                                    18455 non-null  object
 2   Código Empenho Resumido                           18455 non-null  object
 3   Data Emissão                                      18455 non-null  object
 4   Código Tipo Documento                             18455 non-null  object
 5   Tipo Documento                                    18455 non-null  object
 6   Tipo Empenho                                      18455 non-null  object
 7   Espécie Empenho                                   18455 non-null  object
 8   Código Órgão Superior                             18455 non-null  object
 9   Órgão Superior              

In [15]:
# Pode cruzar com todas as despesas se quiser também

In [16]:
despesas_pagamentos_empenho_cruzado = pd.merge(despesas_empenho, despesas_pagamento_empenhosimpactados, left_on='Código Empenho', right_on='Código Empenho')
despesas_pagamentos_empenho_cruzado.shape

(2117981, 70)

In [17]:
#despesas_pagamentos_empenho_cruzado.info()

### Agora com esses pagamentos posso chegar no fornecedor e mais detalhes
### Tenho que abrir o arquivo de pagamentos finais

In [18]:
# Une em um dataframe só os pagamentos finais

In [19]:
folder = Path("zips_empenho_liquidacao_e_pagamento/Despesas_Pagamento")
despesas_pagamentos_finais = pd.concat([
    _reader_pasta(csv)
    for csv in folder.glob("*.csv")
])

In [20]:
despesas_pagamentos_finais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3187467 entries, 0 to 27486
Data columns (total 34 columns):
 #   Column                                Dtype 
---  ------                                ----- 
 0   Código Pagamento                      object
 1   Código Pagamento Resumido             object
 2   Data Emissão                          object
 3   Código Tipo Documento                 object
 4   Tipo Documento                        object
 5   Tipo OB                               object
 6   Extraorçamentário                     object
 7   Código Órgão Superior                 object
 8   Órgão Superior                        object
 9   Código Órgão                          object
 10  Órgão                                 object
 11  Código Unidade Gestora                object
 12  Unidade Gestora                       object
 13  Código Gestão                         object
 14  Gestão                                object
 15  Código Favorecido                 

### Agora o cruzamento é pelo "Código Pagamento"

In [21]:
# Se quiser cruzar com todos os pagamentos
# pagamentos_feitos = pd.merge(despesas_pagamentos_empenho_cruzado, despesas_pagamentos_finais, left_on='Código Pagamento', right_on='Código Pagamento')

# Cruza com pagamentos da covid-19
pagamentos_feitos = pd.merge(despesas_pagamentos_empenho_covid19, despesas_pagamentos_finais, left_on='Código Pagamento', right_on='Código Pagamento')
pagamentos_feitos.shape

(18455, 103)

In [22]:
pagamentos_feitos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18455 entries, 0 to 18454
Columns: 103 entries, Id Empenho to Valor Utilizado na Conversão_y
dtypes: object(103)
memory usage: 14.6+ MB


In [23]:
list(pagamentos_feitos.columns.values.tolist())

['Id Empenho',
 'Código Empenho',
 'Código Empenho Resumido',
 'Data Emissão_x',
 'Código Tipo Documento_x',
 'Tipo Documento_x',
 'Tipo Empenho',
 'Espécie Empenho',
 'Código Órgão Superior_x',
 'Órgão Superior_x',
 'Código Órgão_x',
 'Órgão_x',
 'Código Unidade Gestora_x',
 'Unidade Gestora_x',
 'Código Gestão_x',
 'Gestão_x',
 'Código Favorecido_x',
 'Favorecido_x',
 'Observação_x',
 'Código Esfera Orçamentária',
 'Esfera Orçamentária',
 'Código Tipo Crédito',
 'Tipo Crédito',
 'Código Grupo Fonte Recurso',
 'Grupo Fonte Recurso',
 'Código Fonte Recurso',
 'Fonte Recurso',
 'Código Unidade Orçamentária',
 'Unidade Orçamentária',
 'Código Função',
 'Função',
 'Código SubFunção',
 'SubFunção',
 'Código Programa',
 'Programa',
 'Código Ação',
 'Ação',
 'Linguagem Cidadã',
 'Código Subtítulo (Localizador)',
 'Subtítulo (Localizador)',
 'Código Plano Orçamentário_x',
 'Plano Orçamentário_x',
 'Código Programa Governo_x',
 'Nome Programa Governo_x',
 'Autor Emenda',
 'Código Categoria de 

### O campo "Código Favorecido" é o CNPJ ou CPF mascarado do fornecedor

In [24]:
pagamentos_feitos.to_csv('resultados/pagamentos_covid19/pagamentos_feitos_de_empenhos_covid19_ate_29_junho_2020.csv', index=False)